In [1]:
import numpy as np
import nibabel as nib
import albumentations as A
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

In [2]:
transform = A.Compose(
    p=1.0,
    transforms=[
        A.HorizontalFlip(p=1) # To check if horizontal flip is working
    ]
)

In [3]:
mri = nib.load("../datasets/Task01_BrainTumour/train/images/BRATS_001.nii.gz").get_fdata()
mask = nib.load("../datasets/Task01_BrainTumour/train/masks/BRATS_001.nii.gz").get_fdata().astype(np.uint8)

In [4]:
classes_dict = {
    'Background': 0,
    'Edema': 1,
    'Non-enhancing tumor': 2,
    'Enhancing tumor': 3 
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Edema', 'Non-enhancing tumor', 'Enhancing tumor', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=154, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.title("MRI", fontsize=20)
    plt.imshow(mri[:, :, layer, channel], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask_img = mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask_img)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_img = np.where(mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask_img, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: "FLAIR" Fluid-attenuated inversion recovery
#  Channel 1: "T1w" T1-weighted
#  Channel 2: "t1gd" T1-weighted with gadolinium contrast enhancement
#  Channel 3: "T2w" T2-weighted    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 3))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

In [5]:
transformed = transform(image=mri, mask=mask)
mri = transformed["image"]
mask = transformed["mask"]

In [6]:
mri.dtype

dtype('float64')

In [7]:
mri.shape

(240, 240, 155, 4)

In [8]:
mask.dtype

dtype('uint8')

In [9]:
mask.shape

(240, 240, 155)

In [10]:
classes_dict = {
    'Background': 0,
    'Edema': 1,
    'Non-enhancing tumor': 2,
    'Enhancing tumor': 3 
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Edema', 'Non-enhancing tumor', 'Enhancing tumor', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=154, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.title("MRI", fontsize=20)
    plt.imshow(mri[:, :, layer, channel], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask_img = mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask_img)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_img = np.where(mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask_img, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: "FLAIR" Fluid-attenuated inversion recovery
#  Channel 1: "T1w" T1-weighted
#  Channel 2: "t1gd" T1-weighted with gadolinium contrast enhancement
#  Channel 3: "T2w" T2-weighted    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 3))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…